# 라이브러리

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook
from kiwipiepy import Kiwi

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

kiwi = Kiwi()

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
probability_data = survey[['content_id', 'probability']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
probability_data['content_id'] = probability_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
probability_data = probability_data[probability_data['content_id'] != 0].reset_index().drop(columns=['index'])
probability_data['content_id'] = probability_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(probability_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
probability_data = pd.merge(probability_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
probability_data['probability'] = probability_data['probability'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

probability_data['content'] = probability_data['content'].apply(cleaned_content)

final_df = probability_data[['content', 'probability']]

# 라벨별 개수 확인
print(final_df['probability'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['probability'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['probability'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/960858573.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probability_data['content_id'] = probability_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


probability
 0    4818
-1     145
 1      60
Name: count, dtype: int64


train_data probability
 0    3857
-1     110
 1      51
Name: count, dtype: int64

 test_data probability
 0    984
-1     18
 1      3
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [4]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["probability"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['probability'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './probability_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [5]:
# train 진행
trainer.train() 

  0%|          | 0/2510 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/1626568691.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|█▉        | 500/2510 [01:28<05:50,  5.73it/s]

{'loss': 0.1988, 'learning_rate': 4.00398406374502e-05, 'epoch': 1.99}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/1626568691.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 40%|███▉      | 1000/2510 [02:58<04:16,  5.89it/s]

{'loss': 0.1136, 'learning_rate': 3.00796812749004e-05, 'epoch': 3.98}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/1626568691.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 60%|█████▉    | 1500/2510 [04:26<02:53,  5.84it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/1626568691.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|█████████▉| 2500/2510 [07:21<00:01,  5.72it/s]

{'loss': 0.0632, 'learning_rate': 1.99203187250996e-07, 'epoch': 9.96}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59884/1626568691.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 2510/2510 [07:23<00:00,  5.66it/s]

{'train_runtime': 443.1997, 'train_samples_per_second': 90.32, 'train_steps_per_second': 5.663, 'train_loss': 0.11354910736064987, 'epoch': 10.0}


TrainOutput(global_step=2510, training_loss=0.11354910736064987, metrics={'train_runtime': 443.1997, 'train_samples_per_second': 90.32, 'train_steps_per_second': 5.663, 'train_loss': 0.11354910736064987, 'epoch': 10.0})

In [6]:
trainer.save_model("./probability_model/try_1/probability_model1")
tokenizer.save_pretrained("./probability_model/try_1/probability_tokenizer1")

('./probability_model/try_1/probability_tokenizer1/tokenizer_config.json',
 './probability_model/try_1/probability_tokenizer1/special_tokens_map.json',
 './probability_model/try_1/probability_tokenizer1/vocab.txt',
 './probability_model/try_1/probability_tokenizer1/added_tokens.json')

In [7]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./probability_model/try_1/probability_model1"
tokenizer_path = "./probability_model/try_1/probability_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data


100%|██████████| 1001/1001 [00:11<00:00, 87.63it/s]


,content,probability,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [10]:
print(final_df['probability'].value_counts())

probability
 0    4818
-1     145
 1      60
Name: count, dtype: int64


In [11]:
print(train_data['probability'].value_counts())
print(test_data['probability'].value_counts())

probability
 0    3842
-1     110
 1      51
Name: count, dtype: int64
probability
 0    980
-1     18
 1      3
Name: count, dtype: int64


In [12]:
print(classification_report(test_data['probability'], test_data['pred']))

              precision    recall  f1-score   support

          -1       0.69      0.61      0.65        18
           0       0.99      1.00      1.00       980
           1       0.00      0.00      0.00         3

    accuracy                           0.99      1001
   macro avg       0.56      0.54      0.55      1001
weighted avg       0.98      0.99      0.99      1001



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [13]:
print(classification_report(test_data['probability'], test_data['pred']))
test_data2 = test_data[test_data['probability'] != 0]
print(classification_report(test_data2['probability'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.69      0.61      0.65        18
           0       0.99      1.00      1.00       980
           1       0.00      0.00      0.00         3

    accuracy                           0.99      1001
   macro avg       0.56      0.54      0.55      1001
weighted avg       0.98      0.99      0.99      1001

              precision    recall  f1-score   support

          -1       0.79      0.61      0.69        18
           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         3

    accuracy                           0.52        21
   macro avg       0.26      0.20      0.23        21
weighted avg       0.67      0.52      0.59        21



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [14]:
test = [
    '예측 하나도 안 됨',
    '예측 다 됨',
    '예측성 대박',
    '예측성 최악',
    '예측성 최악',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 예측 하나도 안 됨, 평가: 1
---------- 

문장: 예측 다 됨, 평가: 1
---------- 

문장: 예측성 대박, 평가: 1
---------- 

문장: 예측성 최악, 평가: 1
---------- 

문장: 예측성 최악, 평가: 1
---------- 



## 1차 결과
- 그냥 학습 데이터가 너무 없어서
- 전반적으로 결과 박살남

# 2차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
probability_data = survey[['content_id', 'probability']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
probability_data['content_id'] = probability_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
probability_data = probability_data[probability_data['content_id'] != 0].reset_index().drop(columns=['index'])
probability_data['content_id'] = probability_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(probability_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
probability_data = pd.merge(probability_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
probability_data['probability'] = probability_data['probability'].astype(int)
final_df = probability_data[['content', 'probability']]

# 데이터 추가
new_data = [
    ('개연성 적당함', 1), ('개연성 아주 딱이었음', 1), ('개연성이 알맞아요.', 1),
    ('개연성 완벽했음', 1), ('온도 습도 밝기 서비스 개연성 다 완벽했다', 1), ('개연성 굳', 1),
    ('문제와 스토리와 개연성이 멋짐', 1), ('스토리와 개연성이 끝내줌', 1), ('생각보다 스토리와 개연성이 있었다.', 1), ('스토리도 잘 만들었고 개연성도 좋았다', 1),
    ('개연성 있는 스토리 넘 재밌당', 1), ('스토리와 개연성이 좋은 짱잼 방탈출', 1), ('장치나 자물쇠가 개연성있게 잘 배치되었다.', 1), ('스케일, 개연성, 문제 모든 게 완벽', 1),
    ('개연성이 마땅해서 좋았습니다', 1), ('문제 해결 과정이 개연성 있어서 좋았습니다', 1), ('방 안의 모든 상황이 개연성 있게 이어져서 좋았습니다', 1),
    ('스케일과 개연성이 동시에 완벽하게 구현돼 있어서 감탄했어요', 1), ('전체적으로 방의 구성과 이야기가 완벽한 개연성을 지니고 있었어요', 1),
    ('문제와 스토리가 자연스럽게 이어져서 개연성이 뛰어났습니다', 1), ('스토리의 흐름과 방의 구성이 매끄럽게 이어져서 좋았어요', 1), ('전개가 예상 가능한데도 불구하고 개연성 있는 스토리에 매료됐어요', 1), 
    ('스토리의 전개가 개연성 있게 진행돼서 매우 만족스러웠어요', 1), ('전반적으로 개연성이 높은 방탈출이었습니다', 1),
    ('스케일과 개연성이 동시에 갖춰진 방탈출이었습니다', 1), ('개연성 있는 이야기와 방이 만나 매우 흥미로운 경험이었어요', 1), 
    ('개연성이 뛰어난 스토리에 매료돼서 즐거운 방탈출이었어요', 1), ('방의 각 요소들이 개연성 있게 어울려서 흥미진진한 경험을 했습니다', 1),
    ('이야기와 방이 개연성 있게 이어져서 매우 만족스러웠어요', 1), ('스케일과 개연성이 동시에 완벽하게 표현돼 있어서 감탄했어요', 1),
    ('전개가 예상 가능한데도 불구하고 개연성 있는 스토리에 몰입했습니다', 1), ('스토리의 전개가 개연성 있게 진행돼서 매우 만족스러웠어요', 1),
    ('방탈출 중에도 이야기의 개연성이 높아서 흥미로운 경험이었어요', 1), ('스케일과 개연성이 동시에 갖춰진 방탈출이 기억에 남는 경험이었습니다', 1),
    ('이야기와 방이 끊임없이 개연성 있게 이어져서 기분 좋게 즐겼습니다', 1), ('방의 각 부분이 개연성 있게 설계돼어서 좋은 방탈출이었습니다', 1),

    ('개연성이 없다.', 0),

    ('개연성 개쓰레기임', -1), ('개연성 실화냐', -1), ('말이 전혀 안 되는 시나리오라 다시는 안 가고 싶음', -1), ('돈 받고 이러는 거 아니지', -1),
    ('개연성 개실망.', -1), ('개연성 대실망', -1), ('개연성 최악', -1), ('개연성 무슨 일?', -1), 
    ('개연성 별로임', -1), ('이야기가 안 이어짐', -1), ('개연성 너무 별로임', -1), ('개연성 개별로', -1), ('개연성 정말 쓰레기같아', -1),
    ('개연성 너무 더러움', 1), ('개연성 이상하다 생각함', 1), ('개연성이 살짝 의문', -1), ('문제의 개연성이나 가이드가 부족하다는 느낌을 받음', -1), ('개연성 떨어짐', -1),
    ('개연성을 찾기 어려웠다.', -1),
    ('개연성이 완전히 무너진 상태에서 방을 탈출하는 게 너무 실망스러웠어요', -1), ('이야기가 너무 허술해서 방에서 탈출하는 과정이 하나도 이해 안 갔어요', -1),
    ('돈을 내고 이런 퀄리티의 개연성 없는 시나리오를 본 건 정말 후회스러웠어요', -1), ('개연성이 전혀 없어서 돈주고 입장했다는 게 믿기지 않아요', -1), 
    ('방 전체의 개연성이 너무 부족해서 정말 실망스러웠어요', -1), ('개연성이 전체적으로 대실망 수준이었어요', -1), ('이야기의 흐름이 전혀 이어지지 않아서 개연성이 최악이었어요', -1),
    ('개연성이 너무 부실해서 방에서 벗어나는 게 불가능한 느낌이었어요', -1), ('방 전체의 개연성이 무슨 일이 벌어지는지 이해할 수 없었어요', -1), 
    ('이야기가 전개되지 않아서 개연성이 매우 부족했어요', -1), ('개연성이 전혀 갖춰져 있지 않아서 정말 실망스러웠어요', -1),
    ('방 안의 상황이 전혀 이어지지 않아서 개연성이 정말 쓰레기같았어요', -1), ('개연성이 이상해서 방에서 탈출하는 과정이 이상하게 느껴졌어요', -1),
    ('문제의 개연성이나 가이드가 전혀 부족해서 방탈출이 힘들었어요', -1), ('개연성이 너무 떨어져서 방탈출이 즐거운 경험이 아니었어요', -1), ('개연성을 찾기 어려워서 방에서 탈출하는 데 많이 힘들었어요', -1),
    ('이야기의 전개가 개연성이 없어서 정말 별로였어요', -1), ('방탈출 중에 개연성이 떨어져서 전반적으로 실망스러웠어요', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'probability'])
final_df = pd.concat([final_df, new_df])



# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(cleaned_content)
final_df['content'] = final_df['content'].apply(kiwi_clean)


final_df = final_df.drop_duplicates()
final_df = final_df.reset_index().drop(columns = ['index'])

# 라벨별 개수 확인
print(final_df['probability'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['probability'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['probability'].value_counts())


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_12682/237546230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probability_data['content_id'] = probability_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


probability
 0    4686
-1     179
 1      97
Name: count, dtype: int64


train_data probability
 0    3756
-1     144
 1      70
Name: count, dtype: int64

 test_data probability
 0    901
-1     51
 1     40
Name: count, dtype: int64


In [15]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [16]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["probability"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['probability'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './probability_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [17]:
# train 진행
trainer.train() 

  0%|          | 0/7470 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_258/3605136916.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7470 [01:27<20:13,  5.74it/s]

{'loss': 0.2209, 'learning_rate': 4.6653279785809906e-05, 'epoch': 2.01}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_258/3605136916.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 13%|█▎        | 1000/7470 [02:54<18:17,  5.89it/s]

{'loss': 0.1234, 'learning_rate': 4.3306559571619816e-05, 'epoch': 4.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_258/3605136916.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7470 [04:20<18:13,  5.46it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_258/3605136916.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 94%|█████████▎| 7000/7470 [20:12<01:20,  5.82it/s]

{'loss': 0.0245, 'learning_rate': 3.1459170013386886e-06, 'epoch': 28.11}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_258/3605136916.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7470/7470 [21:33<00:00,  5.78it/s]

{'train_runtime': 1293.3794, 'train_samples_per_second': 92.084, 'train_steps_per_second': 5.776, 'train_loss': 0.0695962799919976, 'epoch': 30.0}


TrainOutput(global_step=7470, training_loss=0.0695962799919976, metrics={'train_runtime': 1293.3794, 'train_samples_per_second': 92.084, 'train_steps_per_second': 5.776, 'train_loss': 0.0695962799919976, 'epoch': 30.0})

In [18]:
trainer.save_model("./probability_model/try_2/probability_model2")
tokenizer.save_pretrained("./probability_model/try_2/probability_tokenizer2")

('./probability_model/try_2/probability_tokenizer2/tokenizer_config.json',
 './probability_model/try_2/probability_tokenizer2/special_tokens_map.json',
 './probability_model/try_2/probability_tokenizer2/vocab.txt',
 './probability_model/try_2/probability_tokenizer2/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./probability_model/try_2/probability_model2"
tokenizer_path = "./probability_model/try_2/probability_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data


100%|██████████| 992/992 [00:10<00:00, 94.61it/s] 


,content,probability,pred
0,탱쫄 판별 가다 가다 판별 메모리 가다 어쨌든 탱으 판별 나다 좋다,0,0
1,인 마지막 방 좋다,0,0
2,인 쏘이다 너무 귀엽다 연출 마지막 연출 인상 깊다 딱 세 가다 좋다 인테리어 너무...,0,0
3,감테 공테 경계선 있다 테마 창공 많다 진짜 무섭다,0,0
4,인생 테마,0,0
...,...,...,...
987,문제 개연 가이드 전혀 부족 방 탈출 힘들다,-1,-1
988,개연 너무 떨어지다 방 탈출 즐겁다 경험 아니다,-1,1
989,개연 찾다 어렵다 방 탈출 많이 힘들다,-1,1
990,이야기 전개 개연 없다 정말 별로 이다,-1,-1


In [6]:
print('0포함', '\n', classification_report(test_data['probability'], test_data['pred']))
test_data2 = test_data[test_data['probability'] != 0]
print('0 제외', '\n', classification_report(test_data2['probability'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.91      0.82      0.87        51
           0       0.99      1.00      1.00       901
           1       0.86      0.90      0.88        40

    accuracy                           0.98       992
   macro avg       0.92      0.91      0.91       992
weighted avg       0.98      0.98      0.98       992

0 제외 
               precision    recall  f1-score   support

          -1       0.98      0.82      0.89        51
           0       0.00      0.00      0.00         0
           1       0.86      0.90      0.88        40

    accuracy                           0.86        91
   macro avg       0.61      0.57      0.59        91
weighted avg       0.92      0.86      0.89        91



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [7]:
test = [
    '개연성 말이 되는 게 없다.',
    '개연성 무슨 일이지...',
    '개연성이 있어서 방탈출이 재미있었어여',
    '개연성 최악',
    '스토리 진짜 좋다.',
    '여기 인테리어 짱이네요',
    '개연성 스레기같아.',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 개연성 말이 되는 게 없다., 평가: 0
---------- 

문장: 개연성 무슨 일이지..., 평가: 0
---------- 

문장: 개연성이 있어서 방탈출이 재미있었어여, 평가: 2
---------- 

문장: 개연성 최악, 평가: 0
---------- 

문장: 스토리 진짜 좋다., 평가: 1
---------- 

문장: 여기 인테리어 짱이네요, 평가: 1
---------- 

문장: 개연성 스레기같아., 평가: 0
---------- 

